<a href="https://colab.research.google.com/github/vishalgimhan/AI_Agents-Projects/blob/main/Multi_Agent_Research_Writer_DeepLearningAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Agents to Research and Write an Article

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 884.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determi

In [1]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

If using OpenAI models

In [5]:
pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=d7b051c8a33eb90a1812ec2a030738e1c9c75639705815abd33482ec30ba4d3a
  Stored in directory: /root/.cache/pip/wheels/15/0c/b3/674aea8c5d91c642c817d4d630bd58faa316724b136844094d
Successfully built utils


In [7]:
# import os
# from utils import get_openai_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_API_KEY"] = openai_api_key

Mistral API

In [ ]:
# OPENAI_API_KEY=your-mistral-api-key
# OPENAI_API_BASE=https://api.mistral.ai/v1
# OPENAI_MODEL_NAME="mistral-small"

Cohere

In [ ]:
# from langchain_community.chat_models import ChatCohere
# # Initialize language model
# os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
# llm = ChatCohere()

# ### you will pass "llm" to your agent function

Hugging Face

In [ ]:
# from langchain_community.llms import HuggingFaceHub

# llm = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
#                      huggingfacehub_api_token="<HF_TOKEN_HERE>",
#                      task="text-generation",
# )

# ### you will pass "llm" to your agent function

Groq

In [32]:
import os
import openai

# llm = openai.OpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     api_key=os.environ.get("GROQ_API_KEY")
# )

# in colab
from google.colab import userdata
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="llama3-8b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY"),
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

### Agent: Planner

In [33]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Writer

In [34]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accuracte "
          "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [35]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks
- Define your tasks, and provide them a `description`, `expected_output` and `agent`.*italicized text*

# Task: Plan

In [36]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

# Task: Write

In [37]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

# Task: Edit

In [38]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [39]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

In [40]:
result = crew.kickoff(inputs={"topic": "Artifiical Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artifiical Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer:

**Content Plan: Artificial Intelligence**

**I. Introduction**

* Title: "Unlocking the Power of Artificial Intelligence: Trends, Players, and Future Prospects"
* Subtitle: "Understanding the latest developments and innovations in AI"
* Keywords: Artificial Intelligence, AI, Machine Learning, Deep Learning, Natural Language Processing
* Meta Description: "Explore the latest trends, key players, and noteworthy news in Artificial Intelligence, and discover the potential applications and future prospect

In [45]:
# Display the results of your execution as markdown in the notebook.
from IPython.display import Markdown
Markdown(result)

**Unlocking the Power of Artificial Intelligence: Trends, Players, and Future Prospects**

## Introduction

Artificial Intelligence (AI) has been a rapidly evolving field in recent years, with significant breakthroughs in machine learning, natural language processing, and computer vision. As AI continues to advance, it's essential to stay informed about the latest trends, key players, and future prospects of this technology. In this article, we'll delve into the latest developments and innovations in AI, exploring the potential applications and future prospects of this technology.

## Latest Trends in AI

Machine learning and deep learning have been driving forces behind many recent breakthroughs in AI. These technologies enable machines to learn from data and improve over time, without being explicitly programmed. Natural language processing and speech recognition have also seen significant advancements, allowing computers to understand and generate human-like language. Computer vision and image recognition have also been improved, enabling machines to interpret and understand visual data.

## Key Players in AI

Several companies and research institutions are at the forefront of AI development. Google's AlphaGo and DeepMind have made significant strides in machine learning and natural language processing, respectively. Facebook's AI Lab and OpenWorm are working on developing AI for healthcare and robotics applications. Amazon's Alexa and SageMaker offer AI-powered voice assistants and machine learning platforms, respectively. Microsoft's Azure Machine Learning and Cognitive Services provide AI-powered cloud computing and data analytics.

## Noteworthy News and Developments

Recent breakthroughs in AI research and applications include the development of AI-powered robots that can perform complex tasks, such as surgery and space exploration. Industry announcements and partnerships have also been made, such as the collaboration between Google and Salesforce to develop AI-powered customer service tools. Government initiatives and regulations have also been put in place to ensure responsible AI development and deployment.

## Future Prospects and Challenges

The potential applications and uses of AI are vast and varied, from healthcare and finance to education and transportation. However, there are also challenges and limitations to AI development, such as the need for large amounts of data and the risk of bias in AI systems. Ethical considerations and potential risks, such as job displacement and privacy concerns, must also be addressed.

## Conclusion

In conclusion, AI has made significant strides in recent years, with advancements in machine learning, natural language processing, and computer vision. The potential applications and uses of AI are vast and varied, and it's essential to stay informed about the latest developments and trends. We invite readers to explore AI further, stay updated on the latest news and breakthroughs, and share their thoughts and experiences with AI.

## Call to Action

* Explore AI further and stay updated on the latest developments and trends.
* Check out relevant articles, courses, and books, such as "Deep Learning" by Ian Goodfellow, Yoshua Bengio, and Aaron Courville, and "AI for Everyone" by Andrew Ng.
* Share your thoughts and experiences with AI, and join the conversation on social media using the hashtag #AI.

Note: The edited article maintains the original structure and content, with minor adjustments to sentence structure and wording to align with the organization's voice and style. The article is free of grammatical errors and provides a balanced perspective on the topic of Artificial Intelligence.